# 1 of 2 Oblivious Transfer (OT)

In a oblivious transfer protocol, a sender wants to send a message $m_0$ or $m_1$ to a receiver. The sender wants the receiver to see only one message and the receiver whants the sender not to know which message he picked. To have a more in deep introduction to oblivious transfer go [here](https://en.wikipedia.org/wiki/Oblivious_transfer).
 
First The sender takes two prime numbers *p* and *q* and calculate *N* as $p*q$

In [1]:
from crypt import RandomPrime, xgcd, fastPowering
from crypt import InverseMod
from random import seed

seed(5)
bits = 16
p = RandomPrime(bits, 40)
q = RandomPrime(bits, 40)
N = p*q

print("Two large primes:\np={}\nq={}\nN=pq={}".format(p, q, N))

Two large primes:
p=46889
q=59263
N=pq=2778782807


The sender calculates $e$ as a random coprime to $(p-1)*(q-1)$ and $d$ as the inverse of $e$ mod (p-1)(q-1). Then the sender publishes the public keys (N, e) and keeps the private key $d$ secret.

In [2]:
from random import randrange

def RSA_choose_e(p, q):
    prime = max(p, q)
    pq_ = (p-1)*(q-1)
    
    while True:
        e = randrange(2, prime)
        g, _, _ = xgcd(e, pq_)
        if g==1:
            return e
e = RSA_choose_e(p, q)
d = InverseMod(e, (p-1)*(q-1))
print("e={}\nd={}".format(e, d))

e=23767
d=525291127


Up to here, we defined the parameters for RSA communication. Now the sender picks two mesages, for instance $m_0=1234$ and $m_1=5678$. He wants to send one and only one to the receiver but without knowing which one the receiver picked.

In [3]:
m = [1234, 4567]

Now, the sender publishes two random messages $x_0$ and $x_1$

In [4]:
x = [randrange(2, N), randrange(2, N)]

print("x0={}\nx1={}".format(x[0], x[1]))

x0=636495402
x1=1460115058


The receiver secrelty flips a coin $b$ that determines which message is going to pick, either 0 or 1. Also generates a random number in between 2 and N and keeps this two values private.

In [5]:
b = randrange(0, 2)
k = randrange(2, N)
print("b={}\nk={}".format(b, k))

b=1
k=2341375824


The receiver computes $v$, this is, $$v = (x_b + k^e)modN$$ and sends this value to the sender

In [6]:
v = (x[b] + fastPowering(k, e, N))%N

print("v={}".format(v))
print((x[b] + fastPowering(k, e, N)))

v=171489492
2950272299


The sender now tries to revert $v$ to get $k$,

$$k_0 = (v - x_0)^d modN$$
$$k_1 = (v - x_1)^d modN$$

Either $k_0$ or $k_1$ is the value of $k$ hat the receiver picked

In [7]:
kl = [pow(v-x[0], d, N), pow(v-x[1], d, N)]

print("k0={}\nk1={}".format(kl[0], kl[1]))

k0=1075015484
k1=2341375824


The sender sends the new masked messages $$m_0^\prime = m_0 + k_0$$ $$m_1^\prime = m_1 + k_1$$ to the receiver

In [8]:
mp = [(m[0]+kl[0]), (m[1]+kl[1])]

print("m0p={}\nm1p={}".format(mp[0], mp[1]))

m0p=1075016718
m1p=2341380391


Then the receiver decrypts the message using his private key $k$.

In [9]:
mb = (mp[b] - k)

print("The message that the receiver decrypted is {}".format(mb))

The message that the receiver decrypted is 4567
